In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import pandas as pd
import os
from tqdm import tqdm

from open_spiel.python.algorithms.exploitability import nash_conv
from open_spiel.python.examples.ubc_decorators import ModalAgentDecorator
from open_spiel.python.examples.ubc_plotting_utils import *
from open_spiel.python.examples.ubc_utils import *

from auctions.webutils import *

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from open_spiel.python.examples.ubc_cma import *

output_notebook()

Loading BokehJS ...

In [4]:
# load runs from experiments
experiments = [
    'jan19_story'
]

runs = []
for experiment in experiments:
    runs += Experiment.objects.get(name=experiment).equilibriumsolverrun_set.all()
print(f"Found {len(runs)} runs")

Found 400 runs


In [5]:
from open_spiel.python.rl_agent import AbstractAgent, StepOutput
class StupidAgent(AbstractAgent):

    def __init__(self, player_id, game):
        self.player_id = player_id
        self.num_actions = game.num_distinct_actions()

    def step(self, time_step, is_evaluation=False):
        if isinstance(time_step, list):
            return [self._act(ts) for ts in time_step]
        else:
            return self._act(time_step)

    def _act(self, time_step):
        if time_step.last():
            return

        state = time_step.observations["state"]
        profits = state.bidders[self.player_id].bidder.get_profits(state.sor_prices[-1])
        legal_actions = time_step.observations["legal_actions"][self.player_id]
        legal_profits = [profits[i] for i in legal_actions]
        
        print('Hi I am a stupid agent')
        print(legal_profits)
        
        action = legal_actions[np.argmin(legal_profits)]

        probs = np.zeros(self.num_actions)
        probs[action] = 1.0
        return StepOutput(action=action, probs=probs)

In [6]:
# TODO: Belongs in script

from open_spiel.python.examples.straightforward_agent import StraightforwardAgent
from open_spiel.python.examples.ubc_decorators import TakeSingleActionDecorator

def get_nash_conv(game, policy):
    worked, time_taken, retval = time_bounded_run(300, nash_conv, game, policy, return_only_nash_conv=False, restrict_to_heuristics=False)
    if worked:
        return retval
    else:
        return None

def get_straightforward_nash_conv(game):
    env_and_policy = make_env_and_policy(game, dict())
    for player in range(game.num_players()):
        env_and_policy.agents[player] = TakeSingleActionDecorator(StraightforwardAgent(player, game), game.num_distinct_actions())
    straightforward_policy = env_and_policy.make_policy()
    return get_nash_conv(game, straightforward_policy)
    
def get_stupid_nash_conv(game):
    env_and_policy = make_env_and_policy(game, dict())
    for player in range(game.num_players()):
        env_and_policy.agents[player] =     (player, game)
    straightforward_policy = env_and_policy.make_policy()
    return get_nash_conv(game, straightforward_policy)
    



# Analyze runs

In [19]:
env_and_policy = make_env_and_policy(game, dict())
for player in range(game.num_players()):
    env_and_policy.agents[player] = TakeSingleActionDecorator(StupidAgent(player, game), game.num_distinct_actions())
straightforward_policy = env_and_policy.make_policy()
straightforward_policy.action_probabilities(game.new_initial_state().child(0).child(0))#.child(3).child(3).child(3).child(3))

{3: 1.0}

In [23]:
env_and_policy.env.envs[0].reset()

TimeStep(observations={'info_state': [array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       0.        , 0.25      , 0.5       , 0.75      , 1.        ,
       0.        , 0.25      , 0.5       , 0.75      , 1.        ,
       0.        , 0.175     , 0.35      , 0.525     , 0.7       ,
       0.3       , 0.475     , 0.65      , 0.825     , 1.        ,
       0.        , 0.10447761, 0.35820895, 0.6119403 , 0.6865672 ,
       0.17910448, 0.5671642 , 0.6567164 , 0.73134327, 0.80597013,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.   

{3: 0.0}

In [20]:
run = runs[0]
game = run.game.load_as_spiel()
nc = get_stupid_nash_conv(game)

KeyError: 'info_dict'

In [94]:
state = game.new_initial_state().child(0).child(0)#.child(3).child(3).child(3).child(3)

In [100]:
agent = StupidAgent(0, game)

In [104]:
agent??

Type:            StupidAgent
String form:     <__main__.StupidAgent object at 0x7f26e22ae2e0>
Docstring:       <no docstring>
Class docstring: Abstract base class for Open Spiel RL agents.
Init docstring: 
Initializes agent.

Args:
  player_id: integer, mandatory. Corresponds to the player position in the
    game and is used to index the observation list.
  session: optional Tensorflow session.
  observation_spec: optional dict containing observation specifications.
  name: string. Must be used to scope TF variables. Defaults to `agent`.
  **agent_specific_kwargs: optional extra args.


In [95]:
iss = state.information_state_string()

In [48]:
br = nc.br_policies[0]

In [98]:
nc.br_policies[1].value(state)

0

In [74]:
br.info_sets?

Signature: br.info_sets(state)
Docstring: Returns a dict of infostatekey to list of (state, cf_probability).
File:      /apps/open_spiel/open_spiel/python/algorithms/best_response.py
Type:      method


In [20]:
env_and_policy = make_env_and_policy(game, dict())
for player in range(game.num_players()):
    env_and_policy.agents[player] = TakeSingleActionDecorator(StupidAgent(player, game), game.num_distinct_actions())
straightforward_policy = env_and_policy.make_policy()

In [38]:
straightforward_policy.

Signature:      straightforward_policy(state, player_id=None)
Type:           JointRLAgentPolicy
String form:    <open_spiel.python.rl_agent_policy.JointRLAgentPolicy object at 0x7f265ec44d30>
File:           /apps/open_spiel/open_spiel/python/rl_agent_policy.py
Docstring:     
Joint policy denoted by the RL agents of a game.

Given a list of RL agents of players for a game, this class can be used derive
the corresponding (joint) policy. In particular, the distribution over
possible actions will be those that are returned by the step() method of
the RL agents given the state.
Init docstring:
Initializes the joint RL agent policy.

Args:
  game: The game.
  agents: Dictionary of agents keyed by the player IDs.
  use_observation: If true then observation tensor will be used as the
    `info_state` in the step() calls; otherwise, information state tensor
    will be used. See `use_observation` property of
    rl_environment.Environment.
Call docstring:
Turns the policy into a callable.

A

In [35]:
ts = env_and_policy.env.reset()
step_output = straightforward_policy.step(ts)
ts = env_and_policy.env.step(step_output[0].action)[0][0]
print(ts)
step_output = agent.step(ts)

AttributeError: 'JointRLAgentPolicy' object has no attribute 'agent'

In [24]:
agent??

Type:            StupidAgent
String form:     <__main__.StupidAgent object at 0x7f270c123160>
Docstring:       <no docstring>
Class docstring: Abstract base class for Open Spiel RL agents.
Init docstring: 
Initializes agent.

Args:
  player_id: integer, mandatory. Corresponds to the player position in the
    game and is used to index the observation list.
  session: optional Tensorflow session.
  observation_spec: optional dict containing observation specifications.
  name: string. Must be used to scope TF variables. Defaults to `agent`.
  **agent_specific_kwargs: optional extra args.


In [19]:
agent.step(game.new_initial_state())

AttributeError: 'ClockAuctionState' object has no attribute 'last'

In [15]:
agent.step(game.new_initial_state())

AttributeError: 'ClockAuctionState' object has no attribute 'last'

In [7]:
records = []
for run in tqdm(runs):    
    game = run.game.load_as_spiel()
    record = {
        'run_name': run.name,
        'game_name': run.game.name, 
        'seed': run.config.get('seed'), 
        'config': run.get_config_name(),
        'alg': get_algorithm_from_run(run),
    }
    
    record.update(get_game_info(game, run.game))  
    
    record['no_error'] = False
    records.append(record) # Put it here so you see the False's in the display
        
    try:
        game, final_checkpoint, policy = get_results(run, load_policy=False)
    except Exception as e:
        print(f"Skipping run {run.name} because of error {e}")
        continue
    
    try:
        
        record['t'] = final_checkpoint.t
        record['walltime'] = run.walltime(),
        evaluation = final_checkpoint.get_modal_eval()
        
        record['nash_conv'] = evaluation.nash_conv
        record['rewards'] = evaluation.mean_rewards
        record['nash_conv_frac'] = evaluation.nash_conv / sum(evaluation.mean_rewards) if not pd.isnull(evaluation.nash_conv) else np.nan
        record['heuristic_conv'] = evaluation.heuristic_conv
        record['heuristic_conv_frac'] = evaluation.heuristic_conv / sum(evaluation.mean_rewards) if not pd.isnull(evaluation.heuristic_conv) else np.nan

        for i in range(game.num_players()):
            record[f'rewards_{i}'] = evaluation.mean_rewards[i]
            record[f'nc_player_improvements_{i}'] = evaluation.nash_conv_player_improvements[i] if not pd.isnull(evaluation.nash_conv) else np.nan
            record[f'nc_player_improvements_frac_{i}'] = (evaluation.nash_conv_player_improvements[i] / evaluation.mean_rewards[i]) if not pd.isnull(evaluation.nash_conv) else np.nan

            record[f'hc_player_improvements_{i}'] = evaluation.heuristic_conv_player_improvements[i] if not pd.isnull(evaluation.heuristic_conv) else np.nan
            record[f'hc_player_improvements_frac_{i}'] = (evaluation.heuristic_conv_player_improvements[i] / evaluation.mean_rewards[i]) if not pd.isnull(evaluation.heuristic_conv) else np.nan

        record.update(**analyze_samples(evaluation.samples, game))

        nc = record['nash_conv']
        hc = record['heuristic_conv']
        # print(f"NashConv = {(np.nan if pd.isnull(nc) else nc):.2f}; HeuristicConv = {(np.nan if pd.isnull(hc) else hc):.2f}")
        
        
        print(get_straightforward_nash_conv(game))
        print(get_stupid_nash_conv(game))
        
        
        record['no_error'] = True
    except Exception as e:
        print(f"Something wrong with {run}. Skipping. {e}")
        # raise e
        # break
        import traceback
        print(traceback.format_exc())

    break

print(len(records))

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

_NashConvReturn(nash_conv=0.0, player_improvements=array([0., 0.]), br_policies=[<open_spiel.python.algorithms.best_response.BestResponsePolicy object at 0x7f0a6b6b2490>, <open_spiel.python.algorithms.best_response.BestResponsePolicy object at 0x7f0a6b6b24f0>])
Something wrong with jan19_jan19_19_base_dev1000_rho0_t4_tie_break-cfr_port_10_extexternal_plus_linear-101 (jan19_story). Skipping. 'info_dict'
Traceback (most recent call last):
  File "/tmp/ipykernel_8528/1951286636.py", line 51, in <module>
    print(get_stupid_nash_conv(game))
  File "/tmp/ipykernel_8528/388578010.py", line 25, in get_stupid_nash_conv
    return get_nash_conv(game, straightforward_policy)
  File "/tmp/ipykernel_8528/388578010.py", line 7, in get_nash_conv
    worked, time_taken, retval = time_bounded_run(300, nash_conv, game, policy, return_only_nash_conv=False, restrict_to_heuristics=False)
  File "/apps/open_spiel/open_spiel/python/examples/ubc_utils.py", line 255, in time_bounded_run
    result = f(*args,

python_clock_auction(filename=jan19/jan19_19_base_dev1000_rho0_t4_tie_break.json)

In [ ]:
df = pd.DataFrame.from_records(records)
with pd.option_context('display.max_rows', None):
    display(df[['value_structure', 'rule', 'base_game_name', 'deviations', 'no_error']].value_counts().sort_index())

In [ ]:
df['heuristic_conv'].plot(kind='hist')
print(df['heuristic_conv'].isnull().sum())

df.query('heuristic_conv.isnull()')[['value_structure', 'rule', 'base_game_name', 'deviations', 'no_error']].value_counts().sort_index()
# Huh???? Why are 0 deviation games showing as null. Is that for real?

In [ ]:
df['nash_conv'].plot(kind='hist')
print(df['nash_conv'].isnull().sum())

df.query('nash_conv.isnull()')[['value_structure', 'rule', 'base_game_name', 'deviations', 'no_error']].value_counts().sort_index()
# Huh???? Why are 0 deviation games showing as null. Is that for real?

In [ ]:
df.groupby(['deviations', 'value_structure'])['auction_lengths'].plot(kind='hist', legend=True)

In [ ]:
palette = dict()
colors = ['red', 'blue', 'magenta', 'green', 'orange', 'brown', 'black', 'navy', 'pink', 'gold', 'darkgreen', 'orangered', 'olive']
# for i, v in enumerate(df['variant'].unique()):
#     palette[v] = colors[i]

In [ ]:
# Need to a) Remove "bad" entries b) Be careful about comparisons that are missing datapoints 
df_plt = df.copy()

In [ ]:
### Remove bad entries

# TODO: generalize to 3+ players

good_thresh = 0.1
# good_thresh_abs = 5
# df_plt = df.query(f'player_improvements_0 < {good_thresh_abs} and player_improvements_1 < {good_thresh_abs}')
df_plt = df.query(f'hc_player_improvements_frac_0 < {good_thresh} and hc_player_improvements_frac_1 < {good_thresh}').copy()
# df_plt = df.query(f'nash_conv_frac < {good_thresh}')
len(df), len(df_plt)

#### So many removed... wow


In [ ]:


# 1) Get max/min for each valuation/treatment pairing over each stat

# First query down to relevant datapoints. Then groupby rule change and SATS =(game_name) and max/min?
metrics = ['total_revenue', 'total_welfare', 'auction_lengths', 'num_lotteries', 'p_lottery', 'exposure_frac']
for i in range(2): # TODO:
    metrics += [f'p{i}_utility', f'p{i}_payment']


df_plt_indexed = df_plt.set_index(['value_structure', 'rule', 'deviations']).sort_index().copy()

def make_data_dict(df):
    data = dict()
    for metric in metrics:
        data[f'max_{metric}'] = df.groupby('base_game_name')[metric].max()
        data[f'min_{metric}'] = df.groupby('base_game_name')[metric].min()
    return pd.DataFrame(data)
    
for idx, grp_df in df_plt.groupby(['value_structure', 'rule', 'deviations']):
    if idx[1] == 'base':
        continue
        
    try:
        data_grp_df = make_data_dict(grp_df)
        normalizer_grp_df = df_plt_indexed.loc[(idx[0], 'base')]
        data_normalized_df = make_data_dict(normalizer_grp_df)

        cmap_norm = plt.matplotlib.colors.TwoSlopeNorm(vmin=0.9, vcenter=1, vmax=1.1)
        cmap = plt.cm.get_cmap('RdBu').copy()
        cmap.set_bad('magenta')
        plt.figure(figsize=(12, 8))
        data = (data_grp_df / data_normalized_df).values.T
        plt.imshow(data, cmap=cmap, norm=cmap_norm)
        plt.title(idx)
        plt.yticks(range(len(data_grp_df.columns)), data_grp_df.columns)
        plt.colorbar()
        plt.show()
    except Exception as e:
        print(idx, e)

In [ ]:
for metric_name in ['auction_lengths', 'num_lotteries', 'unsold_licenses', 'total_revenue', 'total_welfare']:
    plt.figure(figsize=(12, 4))
    sns.swarmplot(data=df, x='base_game_name', y=metric_name, hue='tiebreaking_policy', dodge=True, size=4)
    plt.gca(); plt.legend([], [], frameon=False)
    plt.title(metric_name)
    plt.show()

In [ ]:
plt.figure(figsize=(12, 4))
sns.swarmplot(data=df, x='base_game_name', y='auction_lengths', hue='tiebreaking_policy', dodge=True, size=4)
plt.gca(); plt.legend([], [], frameon=False)

In [ ]:
plt.figure(figsize=(12, 4))
sns.swarmplot(data=df, x='base_game_name', y='num_lotteries', hue='tiebreaking_policy', dodge=True, size=4)
plt.gca(); plt.legend([], [], frameon=False)

In [ ]:
plt.figure(figsize=(12, 4))
sns.swarmplot(data=df, x='base_game_name', y='unsold_licenses', hue='tiebreaking_policy', dodge=True, size=4)
plt.gca(); plt.legend([], [], frameon=False)

In [ ]:
plt.figure(figsize=(12, 4))
sns.swarmplot(data=df, x='base_game_name', y='total_revenue', hue='tiebreaking_policy', dodge=True, size=4)
plt.gca(); plt.legend([], [], frameon=False)

In [ ]:
plt.figure(figsize=(12, 4))
sns.swarmplot(data=df, x='base_game_name', y='total_welfare', hue='tiebreaking_policy', dodge=True, size=4)
plt.gca(); plt.legend([], [], frameon=False)

In [ ]:
g = sns.swarmplot(data=df.query('value_structure == "quasi_linear"'), x='base_game_name', y='total_welfare', hue='tiebreaking_policy', dodge=True, size=4)
plt.gca(); plt.legend([], [], frameon=False)

In [ ]:
# How to get straightforward eval?

records = []
for run in tqdm(runs):    
    game = run.game.load_as_spiel()
    record = {
        'run_name': run.name,
        'game_name': run.game.name, 
        'seed': run.config.get('seed'), 
        'config': run.get_config_name(),
        'alg': get_algorithm_from_run(run),
    }
    
    record.update(get_game_info(game, run.game))  
    
    record['no_error'] = False
    records.append(record) # Put it here so you see the False's in the display
        
    try:
        game, final_checkpoint, policy = get_results(run, load_policy=False)
    except Exception as e:
        print(f"Skipping run {run.name} because of error {e}")
        continue
    
    try:
        
        record['t'] = final_checkpoint.t
        record['walltime'] = run.walltime(),
        evaluation = final_checkpoint.get_straightforward_eval()
        
        record['nash_conv'] = evaluation.nash_conv
        record['rewards'] = evaluation.mean_rewards
        record['nash_conv_frac'] = evaluation.nash_conv / sum(evaluation.mean_rewards) if not pd.isnull(evaluation.nash_conv) else np.nan
        record['heuristic_conv'] = evaluation.heuristic_conv
        record['heuristic_conv_frac'] = evaluation.heuristic_conv / sum(evaluation.mean_rewards) if not pd.isnull(evaluation.heuristic_conv) else np.nan

        for i in range(game.num_players()):
            record[f'rewards_{i}'] = evaluation.mean_rewards[i]
            record[f'nc_player_improvements_{i}'] = evaluation.nash_conv_player_improvements[i] if not pd.isnull(evaluation.nash_conv) else np.nan
            record[f'nc_player_improvements_frac_{i}'] = (evaluation.nash_conv_player_improvements[i] / evaluation.mean_rewards[i]) if not pd.isnull(evaluation.nash_conv) else np.nan

            record[f'hc_player_improvements_{i}'] = evaluation.heuristic_conv_player_improvements[i] if not pd.isnull(evaluation.heuristic_conv) else np.nan
            record[f'hc_player_improvements_frac_{i}'] = (evaluation.heuristic_conv_player_improvements[i] / evaluation.mean_rewards[i]) if not pd.isnull(evaluation.heuristic_conv) else np.nan

        record.update(**analyze_samples(evaluation.samples, game))

        nc = record['nash_conv']
        hc = record['heuristic_conv']
        # print(f"NashConv = {(np.nan if pd.isnull(nc) else nc):.2f}; HeuristicConv = {(np.nan if pd.isnull(hc) else hc):.2f}")
        
        record['no_error'] = True
    except Exception as e:
        print(f"Something wrong with {run}. Skipping. {e}")
        # raise e
        # break
        import traceback
        print(traceback.format_exc())

print(len(records))

In [ ]:
df = pd.DataFrame.from_records(records)
with pd.option_context('display.max_rows', None):
    display(df[['value_structure', 'rule', 'base_game_name', 'deviations', 'no_error']].value_counts().sort_index())

In [ ]:
for metric_name in ['auction_lengths', 'num_lotteries', 'unsold_licenses', 'total_revenue', 'total_welfare']:
    plt.figure(figsize=(12, 4))
    sns.swarmplot(data=df, x='base_game_name', y=metric_name, hue='tiebreaking_policy', dodge=True, size=4)
    plt.gca(); plt.legend([], [], frameon=False)
    plt.title(metric_name)
    plt.show()

In [ ]:
df['nash_conv'] # Would need to compute